Look for possible SLSN events in recent public ZTF alerts:
- Download subset of recent ZTF alerts.
- Filter the subset and ingest.
- Match to redshift catalogs and the BTS surey.
- Run retrained Parsnip classifier

In [ ]:
workdir = './ampel_slsn'   # Assuming this exists

In [ ]:
import requests, os
from astropy.time import Time
import pandas as pd

In [ ]:
# Request archive token from https://ampel.zeuthen.desy.de/live/dashboard/tokens and joining the
# AmpelProject github group.
token = os.environ["ARCHIVE_TOKEN"]
header = {"Authorization": "bearer "+token}
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.RcfFilter import RcfFilter

from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext

In [ ]:
logger = AmpelLogger.get_logger()

## A. Define filter parameters

In [ ]:
# Decent filter parameters
filter_config = {
    "min_ndet": 6,          # Min number of detections
    "max_magpsf": 19.3,      
    "min_dist_to_sso": 15,     
    "min_gal_lat": 7,      
    "min_age": 0.02,
    "max_ipac_age": 100,
    "min_rb": 0,
    "min_drb": 0.99,
}

In [ ]:
t0filter = RcfFilter( **filter_config, logger=logger )
t0filter.post_init()

## B. Getting a handle to alerts.
A _resume token_, which points to a specific set of alerts. We here present three ways of doing this. 

In [ ]:
alert_delta_t = 3.
min_archive_age = 20.
max_archive_age = 100.

In [ ]:
# Adding some handles to make it feasible...
query = {
  "jd": {
    "$gt": Time.now().jd-alert_delta_t,
    "$lt": Time.now().jd
  },
  "candidate": {
    "rb": {
      "$gt": 0.2
    },
    "drb": {
      "$gt": 0.95   
    },
    "magpsf": {
      "$lt": 19.3,
      "$gt": 18
    },
    "ndethist": {
      "$gt": 6,
    },
    "jdstarthist": {
      "$gt": Time.now().jd-max_archive_age,
      "$lt": Time.now().jd-min_archive_age,
    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
# Connection
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'
header = {"Authorization": "bearer "+token}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
response.ok

In [ ]:
resume_token = response.json()['resume_token']

In [ ]:
# Create a vault - the way to input "secrets" into the system
secrets = { 
            "ztf/archive/token": token,
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
# Assume that a configuration file exists here (see Ampel-HU-astro install instructions)
AMPEL_CONF = '../ampel_conf.yaml'

In [ ]:
channel = 'myslsn'

In [ ]:
# Assuming a local MongoDB instance
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'ztf_slsn',
    purge_db = True,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
# Will use NED for spectroscopic redshifts.
cat_conf = {
    'catalogs' : {
        'NEDz' : {
            'use' : 'catsHTM',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['ObjType', 'Velocity', 'z'],
        },
        'LSPhotoZZou' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['photoz','ra','dec','e_photoz','specz','_6','logMassBest','logMassInf','logMassSup'],
            'pre_filter' : None,
            'post_filter' : None,
            'all': False,
        },
        'wiseScosPhotoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto_Corr','ra','dec','wiseID','w1mCorr','w2mCorr'],
            'pre_filter' : None,
            'post_filter' : None,
        },
        'twoMPZ' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['zPhoto','ra','dec','zSpec'],
            'pre_filter' : None,
            'post_filter' : None,
        },
         'PS1_photoz' : {
            'use' : 'extcats',
            'rs_arcsec' : 10.0,
            'keys_to_append' : ['raMean','decMean','z_phot','z_photErr', 'z_phot0', '_2'],
            'pre_filter' : None,
            'post_filter' : None,
        },
    }
}

In [ ]:
# New, new, also with all categories?
ampelz_conf = {
    "max_redshift_category" : 7,
    "t2_dependency": [
        {
            "unit": "T2CatalogMatch",
            "config": cat_conf,
            "link_override": {
                'filter': 'PPSFilter', 'sort': 'jd', "select": "first"
                }
        },
    ]
}

In [ ]:
# Download parsnip model and classifier
r = requests.get('https://box.hu-berlin.de/f/3a19cdb9102e40729737/?dl=1')
if not r.ok:
    print('Model download not successful.')
with open(os.path.join(workdir,'parsnip_model.pt'), 'wb') as fh:
    fh.write(r.content)

In [ ]:
r = requests.get('https://box.hu-berlin.de/f/852129d55b8648b4841b/?dl=1')
if not r.ok:
    print('Classifier download not successful.')
with open(os.path.join(workdir,'parsnip_classifier.pkl'), 'wb') as fh:
    fh.write(r.content)

In [ ]:
parsnip_config = {
    'parsnip_model':os.path.join(workdir,'parsnip_model.pt'), 
    'parsnip_classifier':os.path.join(workdir,'parsnip_classifier.pkl'), 
    "redshift_kind" : 'T2DigestRedshifts',
    "max_ampelz_group" : 7,      # For this purpose we use any available redshift
    "t2_dependency": [
        {
            "unit": "T2DigestRedshifts",
            "config": ampelz_conf,
        },
    ], 
    'plot_suffix': 'png',
    'plot_dir': workdir,
    'tabulator': [
        {'unit': 'ZTFT2Tabulator'},
    ]
}

In [ ]:
directives = [
        {
            "channel": channel,
            "filter": {
                "unit": "RcfFilter",
                "config": filter_config,
                "on_stock_match": 'bypass',
            },
            "ingest": {
                "mux": {
                    "unit": "ZiArchiveMuxer",
                    "config": {
                        "history_days": 999,
                        "future_days": 999
                    },
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                            "state_t2": [
                                {
                                    "unit": "T2DigestRedshifts",
                                    "config": ampelz_conf,
                                },
                                {
                                    "unit": "T2MatchBTS",
                                },
                                {
                                    "unit": "T2RunParsnip",
                                    "config": parsnip_config,
                                },                            
                            ]
                        }
                    ],
                    "insert": {
                        "point_t2": [
                            {
                                'unit': 'T2CatalogMatch',
                                'config': cat_conf,
                                "ingest": {
                                    'filter': 'PPSFilter', 'sort': 'jd', 'select': 'first'
                                }
                            },
                        ],
                    }

                }
            }
        }
    ]

In [ ]:
loader_config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", "stream":response.json()['resume_token']}

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = 1000000000,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZiAlertSupplier",
        'config': {
            'deserialize': None,
            'loader': {
                'unit': 'ZTFArchiveAlertLoader',
                'config': loader_config
            }
        }
    },
    directives = directives
)


In [ ]:
n = ac.run()

In [ ]:
n

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

In [ ]:
table_config =  {
                                        'table_schema' : {
                                            'T2DigestRedshifts': {
                                                'Ampel z' : ['ampel_z'],
                                                'Ampel z group' : ['group_z_nbr'],
                                                'Ampel distance' :['ampel_dist'],
                                            },
                                            'T2MatchBTS' : {
                                                'Type' : ['bts_type'],
                                            },
                                            'T2RunParsnip' : {
                                                'Parsnip(SLSN)' : ['classification','SLSN'],
                                                'Parsnip(SNII)' : ['classification','SNII'],
                                                'Parsnip(SNIa)' : ['classification','SNIa'],
                                                'Parsnip(SNIbc)' : ['classification','SNIbc'],
                                            },                                            
                                        },
                                        'transient_table_schema' : {
                                        },
                                        'include_stock' : True,
                                        'include_channels' : True,
                                        'fmt' : 'csv',
                                        'local_path' : workdir,
                                    }

In [ ]:
t3p = ctx.new_context_unit(
    process_name = "LensPrint",
    unit = "T3Processor",
    execute = [
        {
            "unit": "T3ReviewUnitExecutor",
            "config": {
                "supply": {
                    "unit": "T3DefaultBufferSupplier",
                    "config": {
                        "select": {
                            "unit": "T3FilteringStockSelector",
                            "config": {
                                't2_filter':  {
                                    'unit': 'T2RunParsnip',
                                    'match': {'classification.SLSN': {"$gt":0.5}}
                                }, 
                            }
                        },
                        "load": {
                            "unit": "T3SimpleDataLoader",
                            "config": {
                                "directives": ["STOCK", "T1", "T2DOC"],
                                "channel": channel
                            }
                        }
                     }
                },
                "stage" : {
                    "unit": "T3SimpleStager",
                    "config": {
                        "execute": [
                            {
                                "unit": "TransientTablePublisher",
                                "config": table_config
                            }
                        ]
                    }
                }
            }
        }
    ]
)

In [ ]:
t3p.run()

### Quick inspect exported file

In [ ]:
df = pd.read_csv(os.path.join(workdir,'TransientTable.csv'))

In [ ]:
# Skip events classified by BTS (for simpliticy)
df = df[( (df['Type'].isna()) | (df['Type']=='-') | df['Type'].str.match('SLSN') )]

In [ ]:
df